# Tạo Output thô sơ cho báo cáo chứng chỉ quỹ

In [29]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)

from import_default import *
from import_database import *
from import_other import *

In [30]:
ctv_df = pd.read_excel('../input/ccq/cts02_danh_sach_ctv_ccq.xlsx')
cn_df = pd.read_excel('../input/ccq/cts03_danh_sach_cn_ccq.xlsx')

input_df = pd.read_excel('../input/ccq/cts01_phi_hop_tac_ccq.xlsx')
input_df.columns = [
    "account_no",
    "investor_name",
    "dksh",
    "fund",
    "scheme",
    "trading_date",
    "txn_code",
    "txn_type",
    "ccq_unit",
    "nav",
    "gross_amount",
    "net_amount",
    "order_date",
    "fee_amc",
    "fund_fee",
    "fee_dxx",
    "fee_total",
    "tax",
    "register_partner_source_code",
    "order_partner_source_code",
    "channel_order",
    "bda_name",
    "rm_name",
    "rm_code",
    "branch_code",
    "branch_name_rm",
    "branch_name_account"
]
input_df['dksh'] = input_df['dksh'].astype(str).str.zfill(12)

#### Xử lý OP1

In [31]:
def ten_cn_quan_ly_rm(code):
    try:
        branch_int = int(str(code)[-3:])
        return cn_df[cn_df['ma_cn'] == branch_int]['ten_cn'].iloc[0] if len(cn_df[cn_df['ma_cn'] == branch_int]) > 0 else np.nan
    except (ValueError, IndexError):
        return np.nan
    
def ten_cn_quan_ly_tktt(code):
    try:
        branch_int = int(str(code)[:3])
        return cn_df[cn_df['ma_cn'] == branch_int]['ten_cn'].iloc[0] if len(cn_df[cn_df['ma_cn'] == branch_int]) > 0 else np.nan
    except (ValueError, IndexError):
        return np.nan

In [32]:
op01_chi_tiet_thang = input_df.copy()
op01_chi_tiet_thang['commision'] = op01_chi_tiet_thang.apply(lambda x: 0.0065*x['net_amount'] if x['fund'][:3] == 'VFF' else 0.0115*x['net_amount'], axis=1)
op01_chi_tiet_thang['rm_name'] = op01_chi_tiet_thang['rm_name'].fillna("Dương Thuỳ Linh")
op01_chi_tiet_thang['rm_code_cts'] = op01_chi_tiet_thang['rm_code'].map(ctv_df[['ma_dinh_danh_vnc', 'ma_ctv_tai_cts']].set_index('ma_dinh_danh_vnc')['ma_ctv_tai_cts'].to_dict()).fillna("CTS")
op01_chi_tiet_thang['branch_code'] = op01_chi_tiet_thang['branch_code'].fillna("CTS")
op01_chi_tiet_thang['branch_name_rm'] = op01_chi_tiet_thang['branch_code'].apply(ten_cn_quan_ly_rm)
op01_chi_tiet_thang['branch_name_account'] = op01_chi_tiet_thang['branch_name_account'].apply(ten_cn_quan_ly_tktt)
op01_chi_tiet_thang.index = op01_chi_tiet_thang.index + 1

op01_chi_tiet_thang = op01_chi_tiet_thang[['account_no', 'investor_name', 'dksh', 'fund', 'trading_date', 'txn_code', 'ccq_unit', 'nav', 'gross_amount', 'net_amount', 'commision', 'rm_code_cts', 'rm_name', 'rm_code', 'branch_code', 'branch_name_rm', 'branch_name_account']]

#### Xử lý OP2

#### Xử lý OP3

In [33]:
op03a_ctv_detail = op01_chi_tiet_thang.copy()
op03a_ctv_detail['phan_khuc'] = 'KHCN'
op03a_ctv_detail['chi_nhanh_vietinbank'] = op03a_ctv_detail['branch_name_rm'].fillna(op03a_ctv_detail['branch_name_rm'])
op03a_ctv_detail['ma_cn'] = op03a_ctv_detail['chi_nhanh_vietinbank'].map(cn_df[['ma_cn', 'ten_cn']].set_index('ten_cn')['ma_cn'].to_dict())
op03a_ctv_detail['doanh_thu_ghi_nhan'] = op03a_ctv_detail.apply(lambda x: x['commision']*0.7 if x['rm_code_cts'] == 'CTS' else x['commision']*0.3, axis=1)
op03a_ctv_detail['vung'] = op03a_ctv_detail['ma_cn'].map(cn_df.set_index('ma_cn')['vung'].to_dict())

op03a_ctv_detail = op03a_ctv_detail[['account_no', 'investor_name', 'dksh', 'phan_khuc', 'rm_code_cts', 'ma_cn', 'chi_nhanh_vietinbank', 'net_amount', 'commision', 'doanh_thu_ghi_nhan', 'vung']]
op03a_ctv_detail.index = op03a_ctv_detail.index + 1

#### Xuất dữ liệu

In [34]:
with pd.ExcelWriter('../output/ccq/cts-ccq-report.xlsx', engine='openpyxl') as writer:
    op01_chi_tiet_thang.to_excel(writer, sheet_name='op01_chi_tiet_thang', index=False)
    op03a_ctv_detail.to_excel(writer, sheet_name='op03a_ctv_detail', index=False)


print('✅ Tạo thành công file ../output/ccq/cts-ccq-report.xlsx')

✅ Tạo thành công file ../output/ccq/cts-ccq-report.xlsx
